In [1]:
'''
Archive of old code:

primary_df = mta_turn_sorted.iloc[1:].copy()
less_df = mta_turn_sorted.iloc[:-1]
primary_df['Entries_diff'] = primary_df['ENTRIES'] - less_df['ENTRIES'].values
primary_df['Exits_diff'] = primary_df['EXITS'] - less_df['EXITS'].values

def diff_turnstiles(df):
    df_sorted = df.sort_values(by=['DATE','TIME']).copy()
    df_sorted['entries_diff'] = df_sorted['ENTRIES'].diff()
    df_sorted['exits_diff'] = df_sorted['EXITS'].diff()
    return df_sorted

master_df = pd.DataFrame()
for ca in mta_turn_190330['C/A'].unique():
    for unit in mta_turn_190330['UNIT'].unique():
        for scp in mta_turn_190330['SCP'].unique():
            mta_turn_filtered = pd.DataFrame()
            mta_turn_filtered = mta_turn_190330[(mta_turn_190330['C/A']==ca) & (mta_turn_190330.UNIT==unit) & (mta_turn_190330.SCP==scp)]
            pd.concat([master_df, diff_turnstiles(mta_turn_filtered)])

station_df = mta_turn_190330.groupby(by=['STATION','DATE','TIME']).sum()
station_df['entries_diff'] = station_df['ENTRIES'].diff()
station_df
'''

"\nArchive of old code:\n\nprimary_df = mta_turn_sorted.iloc[1:].copy()\nless_df = mta_turn_sorted.iloc[:-1]\nprimary_df['Entries_diff'] = primary_df['ENTRIES'] - less_df['ENTRIES'].values\nprimary_df['Exits_diff'] = primary_df['EXITS'] - less_df['EXITS'].values\n\ndef diff_turnstiles(df):\n    df_sorted = df.sort_values(by=['DATE','TIME']).copy()\n    df_sorted['entries_diff'] = df_sorted['ENTRIES'].diff()\n    df_sorted['exits_diff'] = df_sorted['EXITS'].diff()\n    return df_sorted\n\nmaster_df = pd.DataFrame()\nfor ca in mta_turn_190330['C/A'].unique():\n    for unit in mta_turn_190330['UNIT'].unique():\n        for scp in mta_turn_190330['SCP'].unique():\n            mta_turn_filtered = pd.DataFrame()\n            mta_turn_filtered = mta_turn_190330[(mta_turn_190330['C/A']==ca) & (mta_turn_190330.UNIT==unit) & (mta_turn_190330.SCP==scp)]\n            pd.concat([master_df, diff_turnstiles(mta_turn_filtered)])\n\nstation_df = mta_turn_190330.groupby(by=['STATION','DATE','TIME']).sum

In [2]:
import numpy as np
import pandas as pd
import pickle
import datetime
import time
import matplotlib.pyplot as plt

In [3]:
#mta_turn_190330 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190330.txt')
#mta_turn_190323 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190323.txt')
#mta_turn_190316 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190316.txt')
#mta_turn_190309 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190309.txt')
#mta_turn_190302 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190302.txt')
mta_turn_all = pd.read_csv('turnstile_data.csv', low_memory=False)

In [4]:
mta_turn_all.rename(columns={mta_turn_all.columns.values[-1]: 'EXITS'}, inplace=True)

In [5]:
mta_turn_all.shape

(3548154, 11)

In [6]:
'''
Remove duplicate headers and pickle the file
'''

def remove_headers(df, header1, load_file):
    if load_file == False:
        drop_rows = [index for index, val in df.iterrows() if val[header1]==header1]
        df.drop(drop_rows, inplace=True)
        # open a file, where you ant to store the data
        file = open('raw_clean', 'wb')
        # dump information to that file
        pickle.dump(df, file)
        # close the file
        file.close()
    else:
        # open a file, where you stored the pickled data
        file = open('raw_clean', 'rb')
        # dump information to that file
        df = pickle.load(file)
        # close the file
        file.close()
    return df


'''
Make a dict that maps times to an integer mapping of time of the day (or standardized time of the day).
We can remove the non-whole number times or we will end up double counting some entries and exits.
'''

def get_day_of_week(date):
    return datetime.datetime.strptime(date, '%m/%d/%Y').weekday()

def get_month(date):
    return datetime.datetime.strptime(date, '%m/%d/%Y').month

def get_day(date):
    return datetime.datetime.strptime(date, '%m/%d/%Y').day

def get_week(date):
    return datetime.datetime.strptime(date, '%m/%d/%Y').isocalendar()[1]

def standard_time(time_str_in):
    time_time = time.strptime(time_str_in, '%H:%M:%S')
    if (time_time.tm_min != 0) or (time_time.tm_sec != 0):
        return np.nan
    else:
        # Logic for standardizing the times to be generally consistent and comparable
        time1 = time.strptime('00:00:00', '%H:%M:%S')
        time2 = time.strptime('04:00:00', '%H:%M:%S')
        time3 = time.strptime('08:00:00', '%H:%M:%S')
        time4 = time.strptime('12:00:00', '%H:%M:%S')
        time5 = time.strptime('16:00:00', '%H:%M:%S')
        time6 = time.strptime('20:00:00', '%H:%M:%S')
        if (time_time >= time1) and (time_time < time2):
            time_str_out = '02:00:00'
        elif (time_time >= time2) and (time_time < time3):
            time_str_out = '06:00:00'
        elif (time_time >= time3) and (time_time < time4):
            time_str_out = '10:00:00'
        elif (time_time >= time4) and (time_time < time5):
            time_str_out = '14:00:00'
        elif (time_time >= time5) and (time_time < time6):
            time_str_out = '18:00:00'
        else:
            time_str_out = '22:00:00'
        return time_str_out

In [8]:
turn_df = remove_headers(mta_turn_all, 'C/A', load_file=False)
turn_df.shape

(3548137, 11)

In [9]:
'''
Cleaning the data
'''
turn_df['entries_int'] = pd.to_numeric(turn_df['ENTRIES'])
turn_df['exits_int'] = pd.to_numeric(turn_df['EXITS'])

In [10]:
turn_df['standard_time'] = turn_df['TIME'].apply(standard_time)

In [11]:
save_clean = True

if save_clean:
    # open a file, where you want to store the data
    file = open('raw_mod', 'wb')
    # dump information to that file
    pickle.dump(turn_df, file)
    # close the file
    file.close()
else:
    # open a file, where you stored the pickled data
    file = open('raw_mod', 'rb')
    # dump information to that file
    turn_df = pickle.load(file)
    # close the file
    file.close()

In [12]:
turn_df.dropna(inplace=True)

In [13]:
station_df = turn_df.groupby(by=['STATION','DATE','standard_time']).sum()
station_df['entries_diff'] = station_df['entries_int'].diff()
station_df['exits_diff'] = station_df['exits_int'].diff()
station_df.reset_index(inplace=True)

In [14]:
'''
NaN out the rows where the station is changing as those values do not make sense
'''
station_chg_index = [i+1 for i, val in enumerate(station_df['STATION'][1:]) if val != station_df['STATION'][i]]
for i in station_chg_index:
    station_df.loc[i, 'entries_diff'] = np.nan
    station_df.loc[i, 'exits_diff'] = np.nan

In [15]:
station_df[station_df['STATION']=='103 ST']['standard_time'].value_counts()

14:00:00    126
06:00:00    126
22:00:00    126
18:00:00    126
02:00:00    126
10:00:00    126
Name: standard_time, dtype: int64

In [16]:
station_df['day_of_week'] = station_df['DATE'].apply(get_day_of_week)
station_df['month'] = station_df['DATE'].apply(get_month)
station_df['day'] = station_df['DATE'].apply(get_day)
station_df['week'] = station_df['DATE'].apply(get_week)

In [17]:
# Drop nan that result from station transition
station_df.dropna(inplace=True)

In [18]:
debug_df = station_df[station_df['STATION']=='103 ST']
error_times = debug_df[debug_df['DATE']=='06/23/2018'].groupby(by='standard_time').sum()[['entries_diff','exits_diff']]

error_df = debug_df[(debug_df['DATE']=='06/23/2018') 
                    & ((debug_df['standard_time']=='10:00:00') 
                       | (debug_df['standard_time']=='06:00:00')
                       | (debug_df['standard_time']=='14:00:00'))]
print(error_df)
error_df['entries_int'].iloc[1] - error_df['entries_int'].iloc[0]

     STATION        DATE standard_time  entries_int  exits_int  entries_diff  \
1470  103 ST  06/23/2018      06:00:00    152333107  598885745         257.0   
1471  103 ST  06/23/2018      10:00:00    151170646  598432802    -1162461.0   
1472  103 ST  06/23/2018      14:00:00    152340924  598890727     1170278.0   

      exits_diff  day_of_week  month  day  week  
1470       608.0            5      6   23    25  
1471   -452943.0            5      6   23    25  
1472    457925.0            5      6   23    25  


-1162461

In [19]:
# Remove negatives and the row after as well (because there is usually a reversal)

def clear_outliers(df, field):
    original_len = df.shape[0]
    clear_list = list(df[df[field] < 0].index)
    clear_list_plus1 = [i+1 for i in clear_list]
    clear_list.append(clear_list_plus1)
    for i in clear_list:
        if i in list(df.index):
            df.loc[i, field] = np.nan
    df.dropna(inplace=True)
    print(str(len(clear_list)) + ' rows dropped from df out of ' + str(original_len) + ' total')
    return df, len(clear_list)
    
station_df, rows_dropped_entries = clear_outliers(station_df, 'entries_diff')
station_df, rows_dropped_entries = clear_outliers(station_df, 'exits_diff')

1946 rows dropped from df out of 265790 total
318 rows dropped from df out of 263845 total


In [20]:
station_df['total_traffic'] = station_df['entries_diff'] + station_df['exits_diff']
grp_station_weekday = station_df.groupby(by=['STATION','day_of_week']).mean()['total_traffic']
grp_station = station_df.groupby(by=['STATION']).mean()['total_traffic']

In [21]:
grp_station.sort_values(ascending=False)

STATION
42 ST-PORT AUTH    2.799034e+07
104 ST             2.650511e+07
3 AV-149 ST        2.450819e+07
72 ST              2.347603e+07
125 ST             2.298854e+07
CANAL ST           1.891696e+07
23 ST              1.715238e+07
18 AV              1.436272e+07
CHAMBERS ST        1.375036e+07
34 ST-HERALD SQ    1.243992e+07
14 ST              1.224272e+07
86 ST              1.202819e+07
KINGS HWY          1.185130e+07
BROOKLYN BRIDGE    1.125392e+07
FRANKLIN AV        1.057534e+07
59 ST              1.046154e+07
FULTON ST          1.014839e+07
190 ST             1.011854e+07
1 AV               1.004532e+07
34 ST-PENN STA     9.956466e+06
COURT SQ           9.423197e+06
59 ST COLUMBUS     8.559343e+06
BAY PKWY           8.070874e+06
183 ST             7.517503e+06
225 ST             7.020078e+06
FLATBUSH AV-B.C    6.861365e+06
47-50 STS ROCK     5.894355e+06
6 AV               5.741863e+06
GRD CNTRL-42 ST    5.218704e+06
LEXINGTON AV/63    5.036421e+06
                       ...     


In [22]:
save_clean = True

if save_clean:
    # open a file, where you want to store the data
    file = open('final', 'wb')
    # dump information to that file
    pickle.dump(station_df, file)
    # close the file
    file.close()
else:
    # open a file, where you stored the pickled data
    file = open('final', 'rb')
    # dump information to that file
    station_df = pickle.load(file)
    # close the file
    file.close()